In [49]:
import scipy.io
import pandas as pd
import numpy as np
import os

main_data_dir = '../data/datasets'
write_data_dir = r'D:\FINKI\40_diploma_thesis\data\datasets_numpy'

In [41]:
signals_df = pd.DataFrame()
consistent_index = None

for data_dir in os.listdir(main_data_dir):
    if data_dir.endswith('.rar'): continue
    for file in os.listdir(f'{main_data_dir}/{data_dir}'):
        if not file.endswith('.mat'): continue
        try:
            # read file
            mat = scipy.io.loadmat(f'{main_data_dir}/{data_dir}/{file}')
            struct = mat[file.strip('.mat')]

            # parse info
            info = struct[0][0][0][0]
            info_dtypes = info.dtype
            info_names = info_dtypes.names
            info_data = info[0]
            info_values = [i[0][0] for i in info_data]
            info_dict = {name: value for (name, value) in zip(info_names, info_values)}

            # parse x
            x = struct[0][0][1]
            x_dtypes = x.dtype
            x_names = x_dtypes.names
            x_data = x[0]
            x_data_shape = x_data.shape
            x_data_dict = dict()

            for i in range(x_data_shape[0]):
                x_data_dict[i] = dict()
                for x_row_element, x_element_name in zip(x_data[i], x_names):
                    try:
                        if x_row_element[0].shape == (1,):
                            x_data_dict[i][f'x_{x_element_name}'] = x_row_element[0][0]
                        else:
                            x_data_dict[i][f'X_{x_element_name}'] = x_row_element[0]
                    except:
                        x_data_dict[i][f'X_{x_element_name}'] = []
            x_data_df = pd.DataFrame(x_data_dict).T

            # parse y
            y = struct[0][0][2]
            y_dtypes = y.dtype
            y_names = y_dtypes.names
            y_data = y[0]
            y_data_shape = y_data.shape
            y_data_dict = dict()

            for i in range(y_data_shape[0]):
                y_data_dict[i] = dict()
                for y_row_element, y_element_name in zip(y_data[i], y_names):
                    try:
                        if y_row_element[0].shape == (1,):
                            y_data_dict[i][f'Y_{y_element_name}'] = y_row_element[0][0]
                        else:
                            y_data_dict[i][f'Y_{y_element_name}'] = y_row_element[0]
                    except:
                        y_data_dict[i][f'Y_{y_element_name}'] = []
            y_data_df = pd.DataFrame(y_data_dict).T

            # parse description
            desc = struct[0][0][3][0][0]
            desc_names = desc.dtype.names
            desc_dict = dict()

            for i in range(len(desc_names)):
                name_struct = struct[0][0][3][0][0][i]
                name_subnames = name_struct.dtype.names
                for element, element_subname in zip(name_struct[0][0], name_subnames):
                    try:
                        if element[0].shape == (1,):
                            desc_dict[f'{desc_names[i]}_{element_subname}'] = element[0][0]
                        else:
                            desc_dict[f'{desc_names[i]}_{element_subname}'] = element[0]
                    except:
                        desc_dict[f'{desc_names[i]}_{element_subname}'] = []
            desc_data_df = pd.Series(desc_dict)

            measurement = pd.Series()
            for row_idx, row in x_data_df.iterrows():
                measurement[f'{row["X_Raster"]}_Data'.replace("HostService", "HostService_64kHz")] = row['X_Data']
            for row_idx, row in y_data_df.iterrows():
                measurement[f'{row["Y_Raster"]}_{row["Y_Name"]}'.replace("HostService", "HostService_64kHz")] = row['Y_Data']

            # assert consistency of index
            if not consistent_index:
                consistent_index = list(measurement.index)
            else:
                assert list(measurement.index) == consistent_index; "Index differs between np arrays"

            np.save(f'{write_data_dir}/{file.strip(".mat")}.npy', measurement.to_numpy())
        except Exception as e:
            print(str(e))



C:\Users\ilija\AppData\Local\Temp/ipykernel_12324/3049034874.py:79: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  measurement = pd.Series()


Expecting matrix here


In [51]:
with open(f'{write_data_dir}/index.txt', 'w') as f:
    f.write(str(list(measurement.index)))